# Практическое задание из раздела 4 - Дообучить модель классификации звука

## Подготовка IDE

### Загрузим необходимые библиотеки

In [1]:
from datasets import load_dataset, Audio
from transformers import AutoFeatureExtractor, TrainingArguments, Trainer, AutoModelForAudioClassification

from huggingface_hub import notebook_login

import evaluate

import numpy as np

## Аутентификация блокнота в HuggingFace

Для того, чтобы иметь возможность сохранить модель на HuggingFace Hub аутентифицируем блокнот:

In [2]:
notebook_login()

## Подготовка датасета

### Описание полей датасета

Типы, связанные с каждым из полей данных, приведены ниже:
- `file`: [string] - путь к соответствующему файлу с данными,
- `audio`: содержит путь к звуковому файлу примера данных, 
- `array`: декодированная форма волны конкретного конкретного примера данных,
- `sampling_rate`: частота дискретизации (семплирования) конкретного примера данных,
- `genre`: признак классификации указывающий жанр конкретного примера данных.

### Загрузка датасет

In [3]:
gtzan = load_dataset("marsyas/gtzan", "all")
gtzan

Found cached dataset gtzan (/home/artyom/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/8bd0e23c2d9b2be30d36bc6834319772dff22a3bd28527996612386cef003910)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})

Разделим датасет на тренировочное и тестовое подмножества. Выделим для тестового подмножества 10% от датасета:

In [4]:
gtzan = gtzan['train'] # Обьект DatasetDict не имеет метода .train_test_split. Преобразуем DatasetDict в объект Dataset
print(type(gtzan))

gtzan = gtzan.train_test_split(seed=42, shuffle=True, test_size=0.1)
gtzan

Loading cached split indices for dataset at /home/artyom/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/8bd0e23c2d9b2be30d36bc6834319772dff22a3bd28527996612386cef003910/cache-52d2398c8e4ac745.arrow and /home/artyom/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/8bd0e23c2d9b2be30d36bc6834319772dff22a3bd28527996612386cef003910/cache-3bcc56e346e4d81c.arrow


<class 'datasets.arrow_dataset.Dataset'>


DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

Посмотрим на один из примеров датасета:

In [5]:
gtzan["train"][7]

{'file': '/home/artyom/.cache/huggingface/datasets/downloads/extracted/26af6d86e0c979b46bcad90a661fea792ae66965860b310bdc366574d25d1457/genres/jazz/jazz.00059.wav',
 'audio': {'path': '/home/artyom/.cache/huggingface/datasets/downloads/extracted/26af6d86e0c979b46bcad90a661fea792ae66965860b310bdc366574d25d1457/genres/jazz/jazz.00059.wav',
  'array': array([-0.05224609,  0.04705811,  0.09838867, ...,  0.05691528,
          0.06399536,  0.07427979]),
  'sampling_rate': 22050},
 'genre': 5}

Частота дескритизации примеров датасета 22050 Гц.

### Предварительная обработка данных

Аудио- и речевые модели требуют, чтобы входные данные были закодированы в формате, который модель может обрабатывать.  преобразование звука во входной формат осуществляется с помощью feature extractor модели. Подобно токенизаторам, 🤗 Transformers предоставляет удобный класс AutoFeatureExtractor, который может автоматически выбирать нужный экстрактор признаков для заданной модели. Начнем с инстанцирования экстрактора признаков для DistilHuBERT из предварительно обученной контрольной точки:

In [6]:
model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

Поскольку частота дискретизации модели и набора данных различна, перед передачей аудиофайла в программу извлечения признаков его необходимо передискретизировать до 16 000 Гц. Для этого сначала нужно получить частоту дискретизации модели от экстрактора признаков:

In [7]:
sampling_rate = feature_extractor.sampling_rate
sampling_rate

16000

#### Передескретизация датасета

Используем метод cast_column для передескретизации частоты примеров датасета:

In [8]:
gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))

Проверим частоту дескретизации прримера из датасета:

In [9]:
gtzan["train"][7]

{'file': '/home/artyom/.cache/huggingface/datasets/downloads/extracted/26af6d86e0c979b46bcad90a661fea792ae66965860b310bdc366574d25d1457/genres/jazz/jazz.00059.wav',
 'audio': {'path': '/home/artyom/.cache/huggingface/datasets/downloads/extracted/26af6d86e0c979b46bcad90a661fea792ae66965860b310bdc366574d25d1457/genres/jazz/jazz.00059.wav',
  'array': array([-0.02912004,  0.06167509,  0.1367783 , ...,  0.04330518,
          0.06990883,  0.05479369]),
  'sampling_rate': 16000},
 'genre': 5}

#### Масштабированием признаков

Для нормальной работы модели нобходимо чтобы все значения признака находились в одном и том же динамическом диапазоне. Это позволит получить стабильность  и сходимость в процессе обучения. Перед тем, как предпринимать какие-либо действия по подготовке примеров датасета к обучению модели посмотрим какие значения средней и дисперсии имеют наши данные. Вычислим их для первого примера:

In [10]:
# Выберем первый пример из датасета
sample = gtzan["train"][0]["audio"]

# Рассчитаем среднее и дисперсию для первого примера
print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

Mean: 0.000185, Variance: 0.0493


Видно, что среднее значение уже близко к нулю, но дисперсия ближе к 0,05. Если бы дисперсия для выборки была больше, это могло бы вызвать проблемы с нашей моделью, так как динамический диапазон аудиоданных был бы очень мал и, следовательно, трудноразделим.
Применим экстрактор признаков и посмотрим, что получится на выходе:

In [11]:
inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])

print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

inputs keys: ['input_values', 'attention_mask']
Mean: -7.45e-09, Variance: 1.0


Cреднее значение теперь очень сильно приближается к нулю, а дисперсия - к единице! Именно в таком виде мы хотим получить наши аудиосэмплы перед подачей их в модель HuBERT.

Определим функцию, которую мы можем применить ко всем примерам в наборе данных. Поскольку мы ожидаем, что длина аудиоклипов будет составлять 30 секунд, мы также будем обрезать все более длинные клипы, используя аргументы max_length и truncation в экстракторе признаков следующим образом:

In [12]:
max_duration = 30.0 # Ограничение на максимальную длительность звука.


def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

Применим ее к набору данных с помощью метода map(). Метод .map() поддерживает работу с пакетами сэмплов, что мы и сделаем, установив batched=True. По умолчанию размер пакета составляет 1000, но мы уменьшим его до 100, чтобы пиковая оперативная память оставалась в разумных пределах. Кроме того, для упрощения обучения мы удалим из набора данных столбцы audio и file.

Примечание: Я произвожу подготовку датасета и обучение модели на локальной машине, поэтому могу увеличить параметр определяющий количество процессов обрабатывающих примеры набора данных num_proc больше 1. 

In [13]:
gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=100,
    num_proc=8,
)
gtzan_encoded

Loading cached processed dataset at /home/artyom/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/8bd0e23c2d9b2be30d36bc6834319772dff22a3bd28527996612386cef003910/cache-eb321a26dd5e68eb_*_of_00008.arrow
Loading cached processed dataset at /home/artyom/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/8bd0e23c2d9b2be30d36bc6834319772dff22a3bd28527996612386cef003910/cache-cf79401fb5431c39_*_of_00008.arrow


DatasetDict({
    train: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 899
    })
    test: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 100
    })
})

#### Переименование столбцов

Для того чтобы Trainer мог обрабатывать метки классов, необходимо переименовать колонку genre в label:

In [14]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")
gtzan_encoded

DatasetDict({
    train: Dataset({
        features: ['label', 'input_values', 'attention_mask'],
        num_rows: 899
    })
    test: Dataset({
        features: ['label', 'input_values', 'attention_mask'],
        num_rows: 100
    })
})

Теперь у нас есть подготовленный к обучению модели датасет. Перейдём к обучению модели.

#### Отображение целочисленных идентификаторов классов в строковые (человекочитаемые)

Чтобы получить от модели человекочитаемый вывод, нам необходимо реализовать функцию перехода (отображения от целочисленных идентификаторов (например, 7) к человекочитаемым меткам классов (например, "поп") и обратно. Для этого можно использовать метод int2str() следующим образом:

In [15]:
id2label_fn = gtzan["train"].features["genre"].int2str
id2label_fn(gtzan["train"][0]["genre"])

id2label = {
    str(i): id2label_fn(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

id2label["7"]

'pop'

## Обучение модели

### Модель классификации аудио DistilHuBert

Сначала нужно загрузить модель для данной задачи. Для этого мы можем использовать класс AutoModelForAudioClassification, который автоматически добавит соответствующую классификационную голову в нашу предварительно обученную модель DistilHuBERT. Инстанцируем модели:

In [16]:
num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.weight', 'projector.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Следующим шагом является определение аргументов обучения, включая размер пакета, шаги накопления градиента, количество эпох обучения и скорость обучения:

In [18]:
model_name = model_id.split("/")[-1] # Получим название модели из её ID
batch_size = 16 # (Нужно пробовать как у https://huggingface.co/Kurokabe/distilhubert-finetuned-gtzan-finetuned-gtzan)
gradient_accumulation_steps = 2
num_train_epochs = 10

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    # fp16=True,
    push_to_hub=True,
)

Определим метрику по которой мы будем оценивать качество модели. Поскольку набор данных сбалансирован, в качестве метрики мы будем использовать accuracy и загружать ее с помощью библиотеки 🤗 Evaluate:

In [19]:
metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

Теперь у нас есть все необходимые компоненты! Давайте инстанцируем Trainer и обучим модель:

In [20]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

# trainer.train()
trainer.train()

/home/artyom/HF_Audio_Course_Practical_tasks/Exercise_4/distilhubert-finetuned-gtzan is already a clone of https://huggingface.co/artyomboyko/distilhubert-finetuned-gtzan. Make sure you pull the latest changes with `repo.git_pull()`.
/home/artyom/.local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

Загрузим лучшую (из полученных в процессе обучения) модель на HuggingFace Hub:

In [21]:
kwargs = {
    "dataset_tags": "marsyas/gtzan",
    "dataset": "GTZAN",
    "model_name": f"{model_name}-finetuned-gtzan",
    "finetuned_from": model_id,
    "tasks": "audio-classification",
}

trainer.push_to_hub(**kwargs)

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file runs/Aug23_22-31-18_MSK-PC-01/events.out.tfevents.1692819085.MSK-PC-01.9812.0:   0%|          | 1.…

To https://huggingface.co/artyomboyko/distilhubert-finetuned-gtzan
   8c17906..dac5b4b  main -> main



## Пробная загрузка модели в HuggingFace Hub

In [ ]:
Загрузим наиболее удачную контрольную точку: 

In [6]:
model = AutoModelForAudioClassification.from_pretrained("distilhubert-finetuned-gtzan/checkpoint-5311/")

In [ ]:
kwargs = {
    "dataset_tags": "marsyas/gtzan",
    "dataset": "GTZAN",
    "repo_id": "artyomboyko",
    "model_name": "distilhubert-finetuned-gtzan",
    "finetuned_from": "ntu-spml/distilhubert",
    "tasks": "audio-classification",
}
model.push_to_hub(**kwargs)